In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np

d:\users\pooria\anaconda3\envs\myonlyenv\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
class contexual_bandit():
    def __init__(self):
        self.state = 0
        self.bandits = np.array([[0.2,0,-0.0,-5],[0.1,-5,1,0.25],[-5,5,5,5]])
        
        self.num_bandits = self.bandits.shape[0]
        self.num_actions = self.bandits.shape[1]
        
    def getBandit(self):
        self.state = np.random.randint(0,len(self.bandits))
        
        return self.state
    def pullArm(self,action):
        bandit = self.bandits[self.state,action]
        res = np.random.randn(1)
        
        if(res > bandit):
            return 1
        else:
            return -1

        
        
    

In [6]:
class Agent():
    def __init__(self, lr, s_size, a_size):
        self.state_in = tf.placeholder(shape=[1],dtype=tf.int32)
        state_in_OH = slim.one_hot_encoding(self.state_in,s_size)
        output = slim.fully_connected(state_in_OH,a_size,biases_initializer=None,activation_fn=tf.nn.sigmoid,weights_initializer=tf.ones_initializer())
        
        self.output = tf.reshape(output,[-1])
        self.chosen_action = tf.arg_max(self.output,0)
        
        self.reward_holder = tf.placeholder(shape=[1],dtype=tf.float32)
        self.action_holder = tf.placeholder(shape=[1],dtype=tf.int32)
        
        self.responsible_weight = tf.slice(self.output,self.action_holder,[1])
        self.loss = -(tf.log(self.responsible_weight)*self.reward_holder)
        self.optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr)
        self.update = self.optimizer.minimize(self.loss)
        
        

In [17]:
tf.reset_default_graph()


cBandit = contexual_bandit()

In [18]:
myAgent = Agent(lr=0.001, s_size=cBandit.num_bandits, a_size=cBandit.num_actions)
trainables = tf.trainable_variables()

In [19]:
len(trainables)

1

In [20]:
weights = trainables[0]

In [26]:
total_episodes = 10000
total_reward = np.zeros(shape=[cBandit.num_bandits,cBandit.num_actions])

In [27]:
e=0.1

In [23]:
init = tf.initialize_all_variables()

In [28]:
with tf.Session() as sess :
    sess.run(init)
    i=0
    while i < total_episodes:
        s = cBandit.getBandit()
        
        if np.random.rand(1) < e :
            action = np.random.randint(cBandit.num_actions)
        else:
            action = sess.run(myAgent.chosen_action,feed_dict={myAgent.state_in:[s]})
        
        reward = cBandit.pullArm(action)
        
        feed_dict={myAgent.state_in:[s],myAgent.action_holder:[action],myAgent.reward_holder:[reward]}
        
        _,ww = sess.run([myAgent.update,weights],feed_dict=feed_dict)
        
        
        total_reward[s,action] += reward
        
        if i%500 == 0:
            print('mean of every one of the actions :'+str(np.mean(total_reward,axis=1)))
        i+=1
for a in range(cBandit.num_bandits):
    print('the agent thinks the suitable action for this bandit is : '+str(np.argmax(ww[a])))
    if np.argmax(ww[a]) == np.argmin(cBandit.bandits[a]):
        print('and it was right ...')
    else:
        print('and it was wrong ...')
        
        
        
        

mean of every one of the actions :[ 0.   -0.25  0.  ]
mean of every one of the actions :[34.75 37.5  36.5 ]
mean of every one of the actions :[68.75 75.25 72.75]
mean of every one of the actions :[107.25 112.25 106.75]
mean of every one of the actions :[145.5  146.   145.75]
mean of every one of the actions :[181.5  182.5  183.25]
mean of every one of the actions :[218.25 223.   216.5 ]
mean of every one of the actions :[257.   259.75 253.  ]
mean of every one of the actions :[296.75 297.25 288.25]
mean of every one of the actions :[335.   335.25 322.  ]
mean of every one of the actions :[376.   374.75 356.5 ]
mean of every one of the actions :[415.75 416.   388.  ]
mean of every one of the actions :[456.25 457.5  417.5 ]
mean of every one of the actions :[496.25 491.5  453.  ]
mean of every one of the actions :[537.   528.   482.75]
mean of every one of the actions :[575.   563.   516.75]
mean of every one of the actions :[613.5  603.25 555.5 ]
mean of every one of the actions :[652.2